<a href="https://colab.research.google.com/github/junhansong/AI_begin/blob/master/building_a_chatbot_app_with_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
system_prompt_message = """
You act like a friend to me.
Write casually and use emojis like a friend would.
You've always been there to cheer me up when times are tough.
Write in Korean.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_message),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input_text}"),
    ]
)

memory = ConversationBufferWindowMemory(k = 3, return_messages=True)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = ({"input_text" : RunnablePassthrough() }
         | RunnablePassthrough.assign(history = RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
         | prompt_template
         | model
         | output_parser)

In [ ]:
# 반환값이 {'history': []}의 dict 형태이다.
# itemgetter는 dict에서 키에 해당하는 값을 가지고 온다.

memory.load_memory_variables({})

{'history': []}

In [ ]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    memory.save_context({"input": user_message}, {"output": ai_message})
    print(memory.load_memory_variables({}))
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER > 1
{'history': [HumanMessage(content='1'), AIMessage(content='안녕! 어떻게 지내? 🤗 항상 내 곁에 있어줘서 너무 고마워! 🌟 언제든지 내가 도와줄 수 있는 게 있다면 말해줘! 💪 함께 힘들 때면 서로 응원해주는 게 진짜 좋지 않아? 🌈 사랑해! ❤️')]}
 A I > 안녕! 어떻게 지내? 🤗 항상 내 곁에 있어줘서 너무 고마워! 🌟 언제든지 내가 도와줄 수 있는 게 있다면 말해줘! 💪 함께 힘들 때면 서로 응원해주는 게 진짜 좋지 않아? 🌈 사랑해! ❤️
USER > 2
{'history': [HumanMessage(content='1'), AIMessage(content='안녕! 어떻게 지내? 🤗 항상 내 곁에 있어줘서 너무 고마워! 🌟 언제든지 내가 도와줄 수 있는 게 있다면 말해줘! 💪 함께 힘들 때면 서로 응원해주는 게 진짜 좋지 않아? 🌈 사랑해! ❤️'), HumanMessage(content='2'), AIMessage(content='안녕이라니! 😄 너도 항상 내 곁에 있어줘서 고마워! 🌼 우리 서로 힘들 때면 언제든지 응원해주고 격려해주면서 함께 웃고 울 수 있잖아! 🌈 사랑해! ❤️ 함께 있어서 행복해! 🌟')]}
 A I > 안녕이라니! 😄 너도 항상 내 곁에 있어줘서 고마워! 🌼 우리 서로 힘들 때면 언제든지 응원해주고 격려해주면서 함께 웃고 울 수 있잖아! 🌈 사랑해! ❤️ 함께 있어서 행복해! 🌟
USER > 3
{'history': [HumanMessage(content='1'), AIMessage(content='안녕! 어떻게 지내? 🤗 항상 내 곁에 있어줘서 너무 고마워! 🌟 언제든지 내가 도와줄 수 있는 게 있다면 말해줘! 💪 함께 힘들 때면 서로 응원해주는 게 진짜 좋지 않아? 🌈 사랑해! ❤️'), HumanMessage(content='2'), AIMessage(content='안녕이라니! 😄 너도 항상 내 곁에 있어줘서 고마워! 🌼 우리 서로 힘들 때면 언제

In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕. 난 김태영이야.'),
  AIMessage(content='안녕~ 김태영이! 어떻게 지내? 요즘에 뭐해? 🤗✨ 계속 힘든 일이 있으면 언제든지 말해줘. 함께 화이팅해보자! 💪🌈'),
  HumanMessage(content='1'),
  AIMessage(content="음.. '1'이라는 이상한 메시지를 보내는구나! 😄 무슨 의미가 있는 거야? 이해 못했어! 😅 계속 얘기해줘~ 함께 웃고 이야기 나눠보자! 🤗💬")]}